## 0. PSEUDOCODE / OVERVIEW

##### Prep data
Merge countries' hazard impact vectors into one gdf.

##### Spatial join to project ADMs
Get centroids of hazard gdf.
<br> For each ADM, spatial join ADM with centroids (contains).

##### Combine into dataframe
Provide unique field names.
<br> Table join on ADM3 code.

## 1. PREPARE WORKSPACE

### 1.1 Load all packages.

In [ ]:
# Built-in:
# dir(), print(), range(), format(), int(), len(), list(), max(), min(), zip(), sorted(), sum(), open(), del, = None, try except, with as, for in, if elif else
# Also: list.append(), list.insert(), list.remove(), count(), startswith(), endswith(), contains(), replace()

import os, sys, glob, re, time, subprocess, string # os.getcwd(), os.path.join(), os.listdir(), os.remove(), time.ctime(), glob.glob(), string.zfill(), string.join()
from os.path import exists # exists()
from functools import reduce # reduce()

import geopandas as gpd # read_file(), GeoDataFrame(), sjoin_nearest(), to_crs(), to_file(), .crs, buffer(), dissolve()
import pandas as pd # .dtypes, Series(), concat(), DataFrame(), read_table(), merge(), to_csv(), .loc[], head(), sample(), astype(), unique(), rename(), between(), drop(), fillna(), idxmax(), isna(), isin(), apply(), info(), sort_values(), notna(), groupby(), value_counts(), duplicated(), drop_duplicates()
from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
from shapely.ops import cascaded_union
from shapely.validation import make_valid  # in apply(make_valid)
import shapely.wkt

import numpy as np # median(), mean(), tolist(), .inf
import fiona, rioxarray # fiona.open()
import rasterio # open(), write_band(), .name, .count, .width, .height. nodatavals, .meta, update(), copy(), write()
from rasterio.plot import show
from rasterio import features # features.rasterize()
from rasterio.features import shapes
from rasterio import mask # rasterio.mask.mask()
from rasterio.enums import Resampling # rasterio.enums.Resampling()
from rasterstats import zonal_stats
from osgeo import gdal, osr, ogr, gdal_array, gdalconst # Open(), SpatialReference, WarpOptions(), Warp(), GetDataTypeName(), GetRasterBand(), GetNoDataValue(), Translate(), GetProjection(), GetAttrValue()

In [2]:
# The usual directories
Project_Fd = os.getcwd()
Current_Fd = os.path.join(Project_Fd, 'Hazard')
Source_Fd = os.path.join(Current_Fd, 'Source', '2022')
Intermed_Fd = os.path.join(Current_Fd, 'Intermediate')

# Auxilliary sources
ADM_Fd = os.path.join(Project_Fd, 'ADM')
ADM_gpkg = os.path.join(ADM_Fd, 'Sahel_AdminBoundaries.gpkg')
CCDR_Fd = 'Q:\GIS\povertyequity\CCDR'

# Check paths
print('\n\n'.join([Project_Fd, Current_Fd, Source_Fd, Intermed_Fd, CCDR_Fd, ADM_Fd, ADM_gpkg]))

Q:\GIS\povertyequity\PTI_Sahel

Q:\GIS\povertyequity\PTI_Sahel\Hazard

Q:\GIS\povertyequity\PTI_Sahel\Hazard\Source\2022

Q:\GIS\povertyequity\PTI_Sahel\Hazard\Intermediate

Q:\GIS\povertyequity\CCDR

Q:\GIS\povertyequity\PTI_Sahel\ADM

Q:\GIS\povertyequity\PTI_Sahel\ADM\Sahel_AdminBoundaries.gpkg


## 2. PREP ADMIN LAYERS

### 2.1 CCDR-to-PTI dictionary

Note: this dictionary was made manually in QGIS based on admin boundary alignments between the incongruent admin area datasets.

CCDR admin area count: (remember that one country is missing in each admin set)
<br>ADM2: 182
<br>ADM3: 1329

PTI admin area count:
<br>ADM2: 275
<br>ADM3: 1433

#### CCDR-PTI dictionary

In [3]:
CCDR_ADM2 = pd.DataFrame(gpd.read_file(
    os.path.join(Intermed_Fd, 'CCDR_ADM2_consolidated.shp')))[['CCDR', 'ADM2_CODE', 'ADM2_s2', 'ADM2_s3', 'ADM2_s4']]

CCDR_ADM3 = pd.DataFrame(gpd.read_file(
    os.path.join(Intermed_Fd, 'CCDR_ADM3_consolidated.shp')))[['CCDR', 'ADM3_CODE', 'ADM3_s2', 'ADM3_s3']]

print(CCDR_ADM2.info(), CCDR_ADM3.info(), '\n\n', CCDR_ADM2.sample(10), '\n\n', CCDR_ADM3.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CCDR       182 non-null    float64
 1   ADM2_CODE  182 non-null    object 
 2   ADM2_s2    23 non-null     object 
 3   ADM2_s3    7 non-null      object 
 4   ADM2_s4    2 non-null      object 
dtypes: float64(1), object(4)
memory usage: 7.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329 entries, 0 to 1328
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CCDR       1329 non-null   float64
 1   ADM3_CODE  1329 non-null   object 
 2   ADM3_s2    4 non-null      object 
 3   ADM3_s3    1 non-null      object 
dtypes: float64(1), object(3)
memory usage: 41.7+ KB
None None 

        CCDR ADM2_CODE ADM2_s2 ADM2_s3 ADM2_s4
130   702.0    TD0702  TD0703    None    None
153  1402.0    TD2102    None    None    None
170  2202.0   

#### Pivot longer: ADM2

In [4]:
Dict_ADM2 = pd.melt(CCDR_ADM2, id_vars=['CCDR'], value_vars=[col for col in CCDR_ADM2 if col.startswith('ADM2')], 
                    value_name='PTI')
Dict_ADM2 = Dict_ADM2.loc[Dict_ADM2['PTI'].notnull()][['CCDR', 'PTI']]
Dict_ADM2

,CCDR,PTI
0,5503.0,BF1103
1,4803.0,BF0403
2,5603.0,BF1203
3,5402.0,BF1002
4,4601.0,BF0201
...,...,...
531,2101.0,TD2103
533,2201.0,TD2201
535,2301.0,TD2004
713,2101.0,TD2104


In [5]:
print('Number of PTI polygons associated with more than one CCDR polygon: ', 
      (len(Dict_ADM2.index.unique()) - len(Dict_ADM2['CCDR'].unique())))
print('Number of CCDR polygons associated with more than one PTI polygon: ', 
      (len(Dict_ADM2.index.unique()) - len(Dict_ADM2['PTI'].unique())))

Number of PTI polygons associated with more than one CCDR polygon:  41
Number of CCDR polygons associated with more than one PTI polygon:  0


#### Pivot longer: ADM3

In [6]:
Dict_ADM3 = pd.melt(CCDR_ADM3, id_vars=['CCDR'], value_vars=[col for col in CCDR_ADM3 if col.startswith('ADM3')], 
                    value_name='PTI')
Dict_ADM3 = Dict_ADM3.loc[Dict_ADM3['PTI'].notnull()][['CCDR', 'PTI']]
Dict_ADM3

,CCDR,PTI
0,550301.0,BF110301
1,480301.0,BF040301
2,560301.0,BF120301
3,540201.0,BF100201
4,540202.0,BF100202
...,...,...
2179,60301.0,ML090401
2199,80401.0,ML040402
2266,100301.0,ML020403
2275,70307.0,ML020304


In [7]:
print('Number of PTI polygons associated with more than one CCDR polygon: ', 
      (len(Dict_ADM3.index.unique()) - len(Dict_ADM3['CCDR'].unique())))
print('Number of CCDR polygons associated with more than one PTI polygon: ', 
      (len(Dict_ADM3.index.unique()) - len(Dict_ADM3['PTI'].unique())))

Number of PTI polygons associated with more than one CCDR polygon:  16
Number of CCDR polygons associated with more than one PTI polygon:  20


## 2. PREP RISK DATA

### 2.1 Clean up risk layers

#### Load all indicator files as dataframes in lists, separated by ADM and country.

In [ ]:
BFA_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "BFA*ADM2*"))]
NER_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "NER*ADM2*"))]
TCD_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "TCD*ADM2*"))]

BFA_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "BFA*ADM2*"))]
MLI_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "MLI*ADM2*"))]
NER_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "NER*ADM2*"))]

In [8]:
DR_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "*DR*ADM2*"))] 
HS_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
           for f in glob.glob(os.path.join(Source_Fd, "*HS*ADM2*"))] 
FL_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
           for f in glob.glob(os.path.join(Source_Fd, "*FL*ADM2*"))] 
LS_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
           for f in glob.glob(os.path.join(Source_Fd, "*LS*ADM2*"))]

DR_ADM3 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "*DR*ADM3*"))] 
HS_ADM3 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
           for f in glob.glob(os.path.join(Source_Fd, "*HS*ADM3*"))] 
FL_ADM3 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
           for f in glob.glob(os.path.join(Source_Fd, "*FL*ADM3*"))] 
LS_ADM3 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
           for f in glob.glob(os.path.join(Source_Fd, "*LS*ADM3*"))]

In [9]:
print(len(DR_ADM2), len(DR_ADM3))
print(len(HS_ADM2), len(HS_ADM3))
print(len(FL_ADM2), len(FL_ADM3))
print(len(LS_ADM2), len(LS_ADM3))

1 3
1 3
5 9
2 5


#### Drought and flood have some inconsistent naming. Rename to match before concatenating.

In [10]:
for df in DR_ADM2:
    print(df.info())
    
for df in DR_ADM3:
    print(df.info()) # Turns out only the first in the ADM3 list is mislabeled.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ADM0_CODE    70 non-null     int64  
 1   ADM0_NAME    70 non-null     object 
 2   ADM1_CODE    70 non-null     int64  
 3   ADM1_NAME    70 non-null     object 
 4   ADM2_CODE    70 non-null     int64  
 5   ADM2_NAME    70 non-null     object 
 6   S1_30p_mean  68 non-null     float64
 7   S1_50p_mean  68 non-null     float64
dtypes: float64(2), int64(3), object(3)
memory usage: 4.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ADM0_CODE   351 non-null    int64  
 1   ADM1_CODE   351 non-null    int64  
 2   ADM1_NAME   351 non-null    object 
 3   ADM2_CODE   351 non-null    int64  
 4   ADM2_NAME   351 non-null    object 
 5   ADM3_CODE   35

In [11]:
DR_ADM3[0] = DR_ADM3[0].rename(columns={'S1_30_mean':'S1_30p_mean', 'S1_50_mean':'S1_50p_mean'})
DR_ADM3[0]

,ADM0_CODE,ADM1_CODE,ADM1_NAME,ADM2_CODE,ADM2_NAME,ADM3_CODE,ADM3_NAME,ADM0_NAME,S1_30p_mean,S1_50p_mean
0,42,55,Plateau-Central,5503,Oubritenga,550301,Ambsouya,Burkina Faso,7.048823,4.407149
1,42,48,Centre-Est,4803,Kourittenga,480301,Andemtenga,Burkina Faso,12.770644,8.473943
2,42,56,Sahel,5603,Soum,560301,Arbinda,Burkina Faso,16.220831,11.226185
3,42,54,Nord,5402,Passoré,540201,Arbollé,Burkina Faso,10.808827,8.088264
4,42,54,Nord,5402,Passoré,540202,Bagaré,Burkina Faso,10.853757,8.082340
...,...,...,...,...,...,...,...,...,...,...
346,42,48,Centre-Est,4801,Boulgou,480112,Zoaga,Burkina Faso,8.407756,5.340776
347,42,54,Nord,5403,Yatenga,540313,Zogoré,Burkina Faso,14.421985,11.696571
348,42,48,Centre-Est,4801,Boulgou,480113,Zonsé,Burkina Faso,10.271879,5.421493
349,42,55,Plateau-Central,5501,Ganzourgou,550107,Zôrgho,Burkina Faso,10.878540,8.333871


In [12]:
print('ADM2 datasets: ', len(FL_ADM2))
for df in FL_ADM2:
    print(df.info())

print('\n\nADM3 datasets: ', len(FL_ADM3))
for df in FL_ADM3:
    print(df.info()) # Turns out only the first in the ADM3 list is mislabeled.

ADM2 datasets:  5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ADM0_CODE       45 non-null     int64  
 1   ADM1_CODE       45 non-null     int64  
 2   ADM2_CODE       45 non-null     int64  
 3   ADM0_NAME       45 non-null     object 
 4   ADM1_NAME       45 non-null     object 
 5   ADM2_NAME       45 non-null     object 
 6   ADM2_pop        45 non-null     float64
 7   RP10_pop_tot    45 non-null     float64
 8   RP100_pop_tot   45 non-null     float64
 9   RP1000_pop_tot  45 non-null     float64
 10  RP10_pop_imp    45 non-null     float64
 11  RP100_pop_imp   45 non-null     float64
 12  RP1000_pop_imp  45 non-null     float64
 13  RP10_EAI        45 non-null     float64
 14  RP100_EAI       45 non-null     float64
 15  RP1000_EAI      45 non-null     float64
 16  pop_EAI         45 non-null     float64
 17  pop_EAI%        45 

In [13]:
FL_ADM3[0] = FL_ADM3[0].rename(columns={'EAE':'EAE_agri'})
FL_ADM3[0]

,ADM0_CODE,ADM1_CODE,ADM2_CODE,ADM3_CODE,ADM1_NAME,ADM2_NAME,ADM3_NAME,ADM0_NAME,ADM3_agri,RP10_agri_C1,...,RP100_agri_C6,RP100_agri_tot,RP1000_agri_C1,RP1000_agri_C2,RP1000_agri_C3,RP1000_agri_C4,RP1000_agri_C5,RP1000_agri_C6,RP1000_agri_tot,EAE_agri
0,42,55,5503,550301,Plateau-Central,Oubritenga,Ambsouya,Burkina Faso,6917.521,352.901,...,214.161,1181.343,334.607,300.907,193.302,110.992,105.341,386.988,1432.136,22.568893
1,42,48,4803,480301,Centre-Est,Kourittenga,Andemtenga,Burkina Faso,27535.256,170.616,...,0.521,746.242,183.862,340.461,274.277,114.314,43.291,20.012,976.217,16.051384
2,42,56,5603,560301,Sahel,Soum,Arbinda,Burkina Faso,16844.342,674.899,...,15.242,2619.194,610.281,1110.368,895.527,428.281,155.942,81.017,3281.415,41.943567
3,42,54,5402,540201,Nord,Passoré,Arbollé,Burkina Faso,23155.195,542.560,...,0.000,1136.122,546.818,646.178,200.124,27.335,8.081,0.000,1428.535,5.454260
4,42,54,5402,540202,Nord,Passoré,Bagaré,Burkina Faso,11681.070,434.851,...,0.000,1353.752,378.008,700.787,491.996,80.486,8.694,1.298,1661.269,10.421354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,42,48,4801,480112,Centre-Est,Boulgou,Zoaga,Burkina Faso,11523.706,42.476,...,4.438,233.442,26.279,60.831,96.339,50.902,18.713,11.482,264.545,7.869606
347,42,54,5403,540313,Nord,Yatenga,Zogoré,Burkina Faso,4431.777,216.279,...,0.000,336.399,226.758,157.750,7.118,0.000,0.000,0.000,391.626,0.019388
348,42,48,4801,480113,Centre-Est,Boulgou,Zonsé,Burkina Faso,13626.980,205.576,...,94.295,979.025,280.893,197.793,290.079,129.277,58.634,176.835,1133.510,33.395475
349,42,55,5501,550107,Plateau-Central,Ganzourgou,Zôrgho,Burkina Faso,20529.832,145.023,...,0.000,494.545,151.909,272.056,159.271,46.421,12.343,0.000,642.000,6.307685


### 2.2 Merge with CCDR ADMs

In [ ]:
CCDR2 = CCDR_ADM2[['CCDR']]

for list_A2 in [DR_ADM2, FL_ADM2, HS_ADM2, LS_ADM2]:
    for df in list_A2:
        df = df.rename(columns={'ADM2_CODE':'CCDR'})
        CCDR2 = CCDR2.merge(df, how='cross')

print(CCDR2.info(), CCDR2.sample(10))

In [ ]:
Indic3 = FL_ADM3 + [CCDR_ADM3] + HS_ADM3 + DR_ADM3 + LS_ADM3
CCDR3 = 
CCDR3 = pd.concat(Indic3, axis=1)
CCDR3

#### Concatenate each group.

In [ ]:
HZ_A2 = {'DR':pd.concat(DR_ADM2, ignore_index=True), 
         'HS':pd.concat(HS_ADM2, ignore_index=True), 
         'FL':pd.concat(FL_ADM2, ignore_index=True),
         'LS':pd.concat(LS_ADM2, ignore_index=True)}
for k in HZ_A2:
    HZ_A2[k] = HZ_A2[k].rename(columns={'ADM2_CODE':'CCDR'}) # Rename admin area field to match CCDR-PTI dictionary.
    print('\n\n', k + ": ", HZ_A2[k]['ADM0_NAME'].unique())
    print('\n\n', HZ_A2[k].info(), '\n\n', HZ_A2[k].sample(10))

In [ ]:
HZ_A3 = {'DR':pd.concat(DR_ADM3, ignore_index=True), 
         'HS':pd.concat(HS_ADM3, ignore_index=True), 
         'FL':pd.concat(FL_ADM3, ignore_index=True),
         'LS':pd.concat(LS_ADM3, ignore_index=True)}
for k in HZ_A3:
    HZ_A3[k] = HZ_A3[k].rename(columns={'ADM3_CODE':'CCDR'})
    print('\n\n', k + ": ", HZ_A3[k]['ADM0_NAME'].unique())
    print('\n\n', HZ_A3[k].info(), '\n\n', HZ_A3[k].sample(10))

#### Remove unnecessary fields.

In [ ]:
# Remove excess variables.
Indicators = ['S1_30p_mean', 'S1_50p_mean', # DR
              'EAE_C4', 'EAE_C4%', # HS
              'builtup_EAI', 'builtup_EAI%', 'pop_EAI', 'pop_EAI%', 'EAE_agri', # FL
              'builtup_tot_exposed', 'pop_tot_exposed' # LS
              ]

Fields = Indicators + ['CCDR']

In [ ]:
Indic_A2 = HZ_A2.copy()
Indic_A3 = HZ_A3.copy()

# Overwrite the values in the dictionary with the updated, smaller dfs.
for k in HZ_A2:
    Indic_A2[k] = HZ_A2[k][HZ_A2[k].columns.intersection(Fields)]
for k in HZ_A3:
    Indic_A3[k] = HZ_A3[k][HZ_A3[k].columns.intersection(Fields)]

Indic_A2

In [ ]:
Indic_A3

#### Rename fields to include hazard type.

In [ ]:
for k in Indic_A2:
    Indic_A2[k].columns = [k + '_' + col if col in Indicators else col for col in Indic_A2[k].columns]
Indic_A2

In [ ]:
for k in Indic_A3:
    Indic_A3[k].columns = [k + '_' + col if col in Indicators else col for col in Indic_A3[k].columns]
Indic_A3

### 2.2 Merge risk indicators by CCDR ADM

#### Merge indicators into one df per ADM type now that hazard indicators have unique labels.

In [ ]:
# No need for dictionary keys anymore. Let's simplify.
Indic_A2 = list(Indic_A2.values())
Indic_A3 = list(Indic_A3.values())
print(len(Indic_A2), len(Indic_A3))

In [ ]:
# Tried doing this as a for loop (for k, prevk in zip()), but was struggling.
Indic_ADM2 = Indic_A2[0].merge(Indic_A2[1], on='CCDR', how='outer')
Indic_ADM2 = Indic_ADM2.merge(Indic_A2[2], on='CCDR', how='outer')
Indic_ADM2 = Indic_ADM2.merge(Indic_A2[3], on='CCDR', how='outer')

Indic_ADM3 = Indic_A3[0].merge(Indic_A3[1], on='CCDR', how='outer')
Indic_ADM3 = Indic_ADM3.merge(Indic_A3[2], on='CCDR', how='outer')
Indic_ADM3 = Indic_ADM3.merge(Indic_A3[3], on='CCDR', how='outer')

In [ ]:
Indic_ADM2.sample(20)

In [ ]:
Indic_ADM3.sample(20)

## 4. MATCH RISK INDICATORS WITH PTI ADMIN AREAS

### 4.1 Categorize indicators by eligible aggregation or disaggregation method.

If splitting a hazard ADM into 2+ project ADMs, AND if indicator does not depend on the polygon size or amount (intensive):
<br> Indicator unchanged, applied to each ADM.

If splitting a hazard ADM into 2+ project ADMs, AND if indicator is dependent on the polygon size or amount (extensive):
<br> No data applied to each ADM.

If dissolving 2+ hazard ADMs into a project ADM, AND if indicator is intensive:
<br> No data applied to each ADM.

If dissolving 2+ hazard ADMs into a project ADM, AND if indicator is extensive:
<br> -- If indicator is count, apply sum.
<br> -- If indicator is average, apply no data.
<br> -- If indicator is max, apply max.

Drought percent: INTENSIVE, AVERAGE
- If >1 CCDR to a PTI, no data.
- If >1 PTI to a CCDR, orig value.

Landslide: EXTENSIVE, COUNT
- If >1 CCDR to a PTI, sum.
- If >1 PTI to a CCDR, no data. 

Flood: EXTENSIVE, COUNT
- If >1 CCDR to a PTI, sum.
- If >1 PTI to a CCDR, no data.

Flood percent: INTENSIVE, PROPORTION
- If >1 CCDR to a PTI, no data.
- If >1 PTI to a CCDR, orig value.

Heat stress: EXTENSIVE, COUNT
- If >1 CCDR to a PTI, sum.
- If >1 PTI to a CCDR, no data.

Heat stress percent: INTENSIVE, PROPORTION
- If >1 CCDR to a PTI, no data.
- If >1 PTI to a CCDR, orig value.

In [ ]:
print(Indic_ADM2.info(), '\n', Indic_ADM3.info())

In [ ]:
# Take the Indicators list and assign intensive or extensive.
Intensive = ['DR_S1_30p_mean', 'DR_S1_50p_mean', # DR
              'HS_EAE_C4%', # HS
              'FL_builtup_EAI%', 'FL_pop_EAI%' # FL
              # LS
            ]

Extensive = [# DR
              'HS_EAE_C4', # HS
              'FL_builtup_EAI', 'FL_pop_EAI', 'FL_EAE_agri', # FL
              'LS_builtup_tot_exposed', 'LS_pop_tot_exposed' # LS
             ]

### 4.2 Group-by aggregations

#### Table join risk indicators onto project ADMs.

In [ ]:
ADM2 = CCDR_ADM2.merge(Indic_ADM2, on='CCDR', how='left')
ADM3 = CCDR_ADM3.merge(Indic_ADM3, on='CCDR', how='left')
print(ADM2.info(), ADM3.info())

#### Identify duplicates and mark them in the expanded dataset.

In [ ]:
# subset rows that have duplicate PTI ADM: dupePTI
dupePTI_A2 = ADM2[ADM2.duplicated(['ADM2_CODE'], keep='first')]
dupePTI_A2 = list(dupePTI_A2['ADM2_CODE'])

dupePTI_A3 = ADM3[ADM3.duplicated(['ADM3_CODE'], keep='first')]
dupePTI_A3 = list(dupePTI_A3['ADM3_CODE'])

In [ ]:
# subset rows that have duplicate CCDR ADM: dupeCCDR
dupeCCDR_A2 = ADM2[ADM2.duplicated(['CCDR'], keep='first')]
dupeCCDR_A2 = list(dupeCCDR_A2['CCDR'])

dupeCCDR_A3 = ADM3[ADM3.duplicated(['CCDR'], keep='first')]
dupeCCDR_A3 = list(dupeCCDR_A3['CCDR'])

In [ ]:
ADM2['dupeCCDR'] = 0 # Default value will be zero
ADM2['dupePTI'] = 0
ADM2.loc[ADM2.CCDR.isin(dupeCCDR_A2), 'dupeCCDR'] = 1
ADM2.loc[ADM2.ADM2_CODE.isin(dupePTI_A2), 'dupePTI'] = 1

ADM3['dupeCCDR'] = 0
ADM3['dupePTI'] = 0
ADM3.loc[ADM3.CCDR.isin(dupeCCDR_A3), 'dupeCCDR'] = 1
ADM3.loc[ADM3.ADM3_CODE.isin(dupePTI_A3), 'dupePTI'] = 1

print(ADM2.sample(10), ADM3.sample(10))

In [ ]:
# Check to make sure there are 2 unique values for the dupe fields.
print(ADM2.nunique(axis=0), ADM3.nunique(axis=0))

In [ ]:
# We can drop CCDR ADM field now
ADM2 = ADM2.drop(columns='CCDR', axis=1)
ADM3 = ADM3.drop(columns='CCDR', axis=1)

#### Group-by PTI ADMs and apply appropriate aggregation or null setting

In [ ]:
# Now we can work with just the PTI ADM features.
ADM2_Int = ADM2.drop(columns=Extensive).groupby(
    'ADM2_CODE', as_index=False).first() # Keep original value

ADM2_Ext = ADM2.drop(columns=Intensive).groupby(
    'ADM2_CODE', as_index=False).sum() # Sum these counts
ADM2_Int.info()

In [ ]:
ADM3_Int = ADM3.drop(columns=Extensive).groupby(
    'ADM3_CODE', as_index=False).first() # Keep original value

ADM3_Ext = ADM3.drop(columns=Intensive).groupby(
    'ADM3_CODE', as_index=False).sum() # Sum these counts
ADM3_Int.info()

In [ ]:
# Change extensive variables to None if there were multiple PTI ADMs
for col in Extensive:
    ADM2_Ext.loc[ADM2_Ext.dupePTI > 0, col] = None
# Change intensive variables to None if there were multiple CCDR ADMs
for col in Intensive:
    ADM2_Int.loc[ADM2_Int.dupeCCDR > 0, col] = None

In [ ]:
for col in Extensive:
    ADM3_Ext.loc[ADM3_Ext.dupePTI > 0, col] = None
for col in Intensive:
    ADM3_Int.loc[ADM3_Int.dupeCCDR > 0, col] = None

In [ ]:
# Merge intensive and extensive variables into single df and save to file.
ADM2_PTI = ADM2_Int.merge(ADM2_Ext, on='ADM2_CODE', how='outer')
ADM2_PTI = ADM2_PTI.loc[:, ~ADM2_PTI.columns.str.contains('dupe')]
ADM3_PTI = ADM3_Int.merge(ADM3_Ext, on='ADM3_CODE', how='outer')
ADM3_PTI = ADM3_PTI.loc[:, ~ADM3_PTI.columns.str.contains('dupe')]
print(ADM2_PTI.info(), ADM3_PTI.info())

In [ ]:
ADM2_PTI.sample(20)

In [ ]:
ADM3_PTI.sample(20)

### 2.1 Concatenate countries' CCDR admin areas into a parent object

In [ ]:
FileNames_ADM2 = [''.join([ISO, '_FL_ADM2_pop_EAI']) for ISO in ['BFA', 'NER', 'TCD']]
FileNames_ADM3 = [''.join([ISO, '_FL_ADM3_pop_EAI']) for ISO in ['BFA', 'MLI', 'NER']]

# ADM2
CCDR_ADM2 = [gpd.read_file(os.path.join(Source_Fd, ''.join([FileName, '.gpkg'])), layer=FileName, driver='GPKG') for FileName in FileNames_ADM2]

# ADM3
CCDR_ADM3 = [gpd.read_file(os.path.join(Source_Fd, ''.join([FileName, '.gpkg'])), layer=FileName, driver='GPKG') for FileName in FileNames_ADM3]

print([Item.info() for Item in CCDR_ADM2], [Item.info() for Item in CCDR_ADM3])

In [ ]:
print('Match all projections for concatenation.')
for prevf, f in zip(CCDR_ADM2, CCDR_ADM2[1:]):
    print('Checking...')
    if f.crs != prevf.crs:
        try:
            f.to_crs(prevf.crs)
            print('Reprojecting to match previous.')
        except:
            pass
    else:
        print('Matches with previous.')
        
print('\n\nFinal CRS list:')
for f in CCDR_ADM2:
    print(f.crs)

In [ ]:
print('Match all projections for concatenation.')
for prevf, f in zip(CCDR_ADM3, CCDR_ADM3[1:]):
    print('Checking...')
    if f.crs != prevf.crs:
        try:
            f.to_crs(prevf.crs)
            print('Reprojecting to match previous.')
        except:
            pass
    else:
        print('Matches with previous.')
        
print('\n\nFinal CRS list:')
for f in CCDR_ADM3:
    print(f.crs)

In [ ]:
CCDR_A2 = pd.concat(CCDR_ADM2, ignore_index=True)
print(CCDR_A2.info(), '\n\n', CCDR_A2['ADM0_NAME'].unique(), '\n\n', CCDR_A2.head(10))

In [ ]:
CCDR_A3 = pd.concat(CCDR_ADM3, ignore_index=True)
print(CCDR_A3.info(), '\n\n', CCDR_A3['ADM0_NAME'].unique(), '\n\n', CCDR_A3.head(10))

In [ ]:
# Drop excess variables and rename the ADM codes since their names are identical to the PTI.
CCDR_A2 = CCDR_A2[['ADM0_CODE', 'ADM2_CODE', 'geometry']]
CCDR_A2 = CCDR_A2.rename(columns={'ADM0_CODE':'ISO', 'ADM2_CODE':'ADM2_CCDR'})

CCDR_A3 = CCDR_A3[['ADM0_CODE', 'ADM3_CODE', 'geometry']]
CCDR_A3 = CCDR_A3.rename(columns={'ADM0_CODE':'ISO', 'ADM3_CODE':'ADM3_CCDR'})

print(CCDR_A2.info(), CCDR_A3.info())